*   분류 : 로지스틱회귀
*   회귀 : 선형회귀(다항회귀 변경 - poly 추가)
*   교차검증, 폴드를 따로 지정해서 적용
*   그리드서치, 랜덤서치



데이콘 문제 풀기

In [4]:
!unzip '/content/웹로그기반조회수예측.zip'

Archive:  /content/웹로그기반조회수예측.zip
   creating: open/
  inflating: open/sample_submission.csv  
  inflating: open/test.csv           
  inflating: open/train.csv          


In [5]:
import pandas as pd

In [6]:
df = pd.read_csv('/content/open/train.csv')
df.head(1)

,sessionID,userID,TARGET,browser,OS,device,new,quality,duration,bounced,transaction,transaction_revenue,continent,subcontinent,country,traffic_source,traffic_medium,keyword,referral_path
0,SESSION_000000,USER_000000,17.0,Chrome,Macintosh,desktop,0,45.0,839.0,0,0.0,0.0,Americas,Northern America,United States,google,organic,Category8,NaN


In [7]:
X = df.loc[:,'browser':]
y = df['TARGET']
X.head(1)

,browser,OS,device,new,quality,duration,bounced,transaction,transaction_revenue,continent,subcontinent,country,traffic_source,traffic_medium,keyword,referral_path
0,Chrome,Macintosh,desktop,0,45.0,839.0,0,0.0,0.0,Americas,Northern America,United States,google,organic,Category8,NaN


In [8]:
test_df = pd.read_csv('/content/open/test.csv')
test_df = test_df[ X.columns]
test_df.head(1)

,browser,OS,device,new,quality,duration,bounced,transaction,transaction_revenue,continent,subcontinent,country,traffic_source,traffic_medium,keyword,referral_path
0,Chrome,Linux,desktop,1,75.0,698.0,0,0.0,0.0,Americas,Northern America,United States,(direct),(none),NaN,Category1


In [9]:
X.isna().sum()

browser                     0
OS                          0
device                      0
new                         0
quality                     0
duration                    0
bounced                     0
transaction                 0
transaction_revenue         0
continent                   0
subcontinent                0
country                     0
traffic_source              0
traffic_medium              0
keyword                137675
referral_path          161107
dtype: int64

In [10]:
test_df.isna().sum()

browser                    0
OS                         0
device                     0
new                        0
quality                    0
duration                   0
bounced                    0
transaction                0
transaction_revenue        0
continent                  0
subcontinent               0
country                    0
traffic_source             0
traffic_medium             0
keyword                43070
referral_path          53891
dtype: int64

In [11]:
# # 전처리 - 최빈값으로 전처리
# temp = X.isna().mean()
# for na_obj_col in temp[temp>0].index:
#   X[na_obj_col] = X[na_obj_col].fillna(X[na_obj_col].mode()[0])

# temp = test_df.isna().mean()
# for na_obj_col in temp[temp>0].index:
#   test_df[na_obj_col] = test_df[na_obj_col].fillna(test_df[na_obj_col].mode()[0])

# 전처리 - 임이의 빈 값으로 전처리
temp = X.isna().mean()
for na_obj_col in temp[temp>0].index:
  X[na_obj_col] = X[na_obj_col].fillna('na')

temp = test_df.isna().mean()
for na_obj_col in temp[temp>0].index:
  test_df[na_obj_col] = test_df[na_obj_col].fillna('na')


In [12]:
# 2.전처리 - object를 수치형으로 변경
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

object_colunms = X.select_dtypes(include=['object']).columns
object_colunms

Index(['browser', 'OS', 'device', 'continent', 'subcontinent', 'country',
       'traffic_source', 'traffic_medium', 'keyword', 'referral_path'],
      dtype='object')

In [13]:
X.describe(include='object')

,browser,OS,device,continent,subcontinent,country,traffic_source,traffic_medium,keyword,referral_path
count,252289,252289,252289,252289,252289,252289,252289,252289,252289,252289
unique,70,21,3,6,23,205,168,7,624,1579
top,Chrome,Windows,desktop,Americas,Northern America,United States,google,organic,na,na
freq,174395,88119,167133,126506,111945,104201,116007,107370,137675,161107


In [14]:
for colname in object_colunms:
  try:
    np.unique(test_df[colname])
  except:
    print(colname)

browser
OS
device
continent
subcontinent
country
traffic_source
traffic_medium
keyword
referral_path


In [15]:
# 학습용데이터를 기준으로 라벨인코딩을하는데 검증용데이터중에 추가되는 범주형 데이터가 있으면 라벨인코더에 추가한다
import numpy as np
for colname in object_colunms:
  le.fit(X[colname])
  X[colname] = le.transform(X[colname])

  for case_ in np.unique(test_df[colname]):
    if case_ not in le.classes_:
      le.classes_ = np.append(le.classes_,case_)

  test_df[colname]  =  le.transform(test_df[colname])


In [16]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(X)
X_scaled = ss.transform(X)
X_test_scaled = ss.transform(test_df)

In [17]:
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_validate



lr = LinearRegression();ridge = Ridge();lasso = Lasso()
dtree = DecisionTreeRegressor();
# knr = KNeighborsRegressor()
print(cross_validate(lr,X_scaled,y)['test_score'].mean())
print(cross_validate(ridge,X_scaled,y)['test_score'].mean())
print(cross_validate(lasso,X_scaled,y)['test_score'].mean())
print(cross_validate(dtree,X_scaled,y)['test_score'].mean())
# print(cross_validate(knr,X_scaled,y)['test_score'].mean())

0.5770001943898533
0.5770002814718613
0.5157652805225847
0.5213792864333219


In [18]:
from xgboost import XGBRegressor,XGBRFRegressor
from lightgbm import LGBMRegressor,DaskLGBMRegressor
from sklearn.ensemble import VotingRegressor,BaggingRegressor,AdaBoostRegressor,StackingRegressor,ExtraTreesRegressor,RandomForestRegressor,GradientBoostingRegressor,HistGradientBoostingRegressor

In [19]:
# xgbb,xgbrf = XGBRegressor(),XGBRFRegressor()
# lbgmr = LGBMRegressor(force_col_wise=True)
# print(cross_validate(xgbb,X_scaled,y)['test_score'].mean())
# print(cross_validate(xgbrf,X_scaled,y)['test_score'].mean())
# print(cross_validate(lbgmr,X_scaled,y)['test_score'].mean())

In [ ]:
# for model in [ExtraTreesRegressor(),RandomForestRegressor(),GradientBoostingRegressor(),HistGradientBoostingRegressor()]:
#   print(cross_validate(model,X_scaled,y)['test_score'].mean())

In [21]:
best_model = LGBMRegressor(force_col_wise=True)
best_model.fit(X_scaled,y)

[LightGBM] [Info] Total Bins 1177
[LightGBM] [Info] Number of data points in the train set: 252289, number of used features: 16
[LightGBM] [Info] Start training from score 3.592626


LGBMRegressor(force_col_wise=True)

In [ ]:

# # 파이퍼 파라메터 튜닝
# from sklearn.model_selection import GridSearchCV
# param_grid = {
#     'learning_rate': [0.05, 0.1, 0.2],
#     'n_estimators': [100, 500, 1000],
#     'num_leaves': [31, 50, 100],
#     'max_depth': [-1, 5, 10],
#     'min_child_samples': [20, 50, 100],
#     'subsample': [0.8, 0.9, 1.0],
#     'colsample_bytree': [0.8, 0.9, 1.0],
#     'reg_alpha': [0.1, 0.5, 1.0],
#     'reg_lambda': [0.1, 0.5, 1.0]
# }
# search = GridSearchCV(LGBMRegressor(),param_grid=param_grid ,n_jobs=-1 )
# search.fit(X_scaled,y)
# best_model = search.best_estimator_

In [22]:
predict = best_model.predict(X_test_scaled)
submission_df = pd.read_csv('/content/open/sample_submission.csv')
submission_df['TARGET'] = predict
submission_df.to_csv("제출.csv",index=False)

Voting
  - 여러모델중에 높은성능을 보이는 모델에 투표를 해서 평가

In [ ]:
!pip install catboost
from catboost import CatBoostRegressor

In [27]:
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [29]:
from sklearn.ensemble import VotingRegressor
vote_clf = VotingRegressor(
    estimators=[ ('catbosst',CatBoostRegressor()),('xgb',XGBRegressor()),('lgb',LGBMRegressor())  ]
)

In [ ]:
vote_clf.fit(X_scaled,y)

In [32]:
predict = vote_clf.predict(X_test_scaled)
submission_df = pd.read_csv('/content/open/sample_submission.csv')
submission_df['TARGET'] = predict
submission_df.to_csv("제출2.csv",index=False)

In [33]:
gbr = GradientBoostingRegressor()
crv =cross_validate(gbr,X_scaled,y, return_train_score=True)
crv

{'fit_time': array([18.81723094, 17.84079957, 18.8724227 , 18.11762428, 17.90701199]),
 'score_time': array([0.05209589, 0.05253434, 0.05297375, 0.05151415, 0.05354834]),
 'test_score': array([0.67174008, 0.75067748, 0.74282857, 0.71121618, 0.72398044]),
 'train_score': array([0.77062014, 0.76496576, 0.77039984, 0.77450302, 0.7721482 ])}

In [34]:
np.mean(crv['train_score']), np.mean(crv['test_score'])

(0.7705273887001631, 0.7200885501746097)

In [36]:
gbr.fit(X_scaled,y)

GradientBoostingRegressor()

In [37]:
predict = gbr.predict(X_test_scaled)
submission_df = pd.read_csv('/content/open/sample_submission.csv')
submission_df['TARGET'] = predict
submission_df.to_csv("제출3.csv",index=False)